In [1]:
import common
import importlib
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import socket

# Show all columns and rows in a dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Performance of each platform

In [2]:
# graphalytics inserts
data_ins = common.import_gfe("view_graphalytics_inserts").query("omp_threads.isnull() and cluster == 'stones2' and num_threads_read == 0").copy() # data from the experiments
data_ins["build_frequency"].fillna(pd.Timedelta(0), inplace=True) # replace NaT with 0, otherwise the records are ignored in the group by

# graphalytics load
data_load = common.import_gfe("view_graphalytics_load").query("omp_threads.isnull() and cluster == 'stones2' and num_threads_read == 0").copy()
# shape the dataframe as data_ins
data_load["build_frequency"] = pd.Timedelta(0)
data_load["build_frequency_secs"] = np.NaN
data_load["num_snapshots_created"] = 0
data_load["num_threads_write"] = 0

data = pd.concat([data_ins, data_load])
data["compiler"] = data["compiler_family"].fillna("n/a")

Integrate with the results of GraphMat and GraphBlas

In [3]:
graphmat_weighted = common.import_graphmat("data/graphmat/stones201_weighted.csv").copy()
graphmat_weighted["t_processing"] = graphmat_weighted["t_processing"].apply(lambda x: x.total_seconds())
graphmat_weighted = graphmat_weighted.groupby(["graph", "algorithm"]).agg(completion_time=("t_processing", "median"), count=("t_processing", "count"))
graphmat_weighted = graphmat_weighted.unstack(1)["completion_time"]
graphmat_unweighted = common.import_graphmat("data/graphmat/stones201_unweighted.csv").copy()
graphmat_unweighted["t_processing"] = graphmat_unweighted["t_processing"].apply(lambda x: x.total_seconds())
graphmat_unweighted = graphmat_unweighted.groupby(["graph", "algorithm"]).agg(completion_time=("t_processing", "median"), count=("t_processing", "count"))
graphmat_unweighted = graphmat_unweighted.unstack(1)["completion_time"]
graphmat = pd.concat([
    graphmat_unweighted.loc[["graph500-24", "uniform-24"], ["BFS", "CDLP", "LCC", "PR", "WCC"]], 
    graphmat_weighted.loc[["graph500-24", "uniform-24"], ["SSSP"]]], 
axis=1)
graphmat = graphmat.rename({"BFS": "bfs", "CDLP": "cdlp", "LCC": "lcc", "PR": "pagerank", "SSSP": "sssp", "WCC": "wcc"}, axis=1)
graphmat["library"] = "graphmat"
graphmat["compiler"] = "n/a"
graphmat["build_frequency"] = pd.Timedelta(0)
graphmat["num_threads_read"] = graphmat["num_threads_write"] = 0
graphmat.set_index(["library", "compiler", "build_frequency", "num_threads_read", "num_threads_write"], append=True, inplace=True)
graphmat = graphmat.reorder_levels([1,2,0,3,4,5])
graphblas = common.import_graphmat("data/graphblas/stones216.csv").copy() # same format of graphmat
graphblas["t_processing"] = graphblas["t_processing"].apply(lambda x: x.total_seconds())
graphblas = graphblas.groupby(["graph", "algorithm"]).agg(completion_time=("t_processing", "median"), count=("t_processing", "count"))
graphblas = graphblas.unstack(1)["completion_time"]
graphblas.rename({"BFS": "bfs", "CDLP": "cdlp", "LCC": "lcc", "PR": "pagerank", "SSSP": "sssp", "WCC": "wcc"}, axis=1, inplace=True)
graphblas["library"] = "graphblas"
graphblas["compiler"] = "n/a"
graphblas["build_frequency"] = pd.Timedelta(0)
graphblas["num_threads_read"] = graphblas["num_threads_write"] = 0
graphblas.set_index(["library", "compiler", "build_frequency", "num_threads_read", "num_threads_write"], append=True, inplace=True)
graphblas = graphblas.reorder_levels([1,2,0,3,4,5])

In [4]:
analysis = data.groupby(["library", "compiler", "graph", "build_frequency", "num_threads_read", "num_threads_write", "algorithm"]) \
    .agg(completion_time=("median_secs", "median"), count=("median_secs", "count"))
analysis = pd.concat( [analysis.unstack("algorithm")[("completion_time")], graphblas, graphmat ] )

In [5]:
analysis

bfs  \
library                  compiler graph       build_frequency num_threads_read num_threads_write               
baseline                 n/a      dota-league 0 days 00:00:00 0                40                   6.902463   
                                  graph500-22 0 days 00:00:00 0                40                  18.939507   
                                  graph500-24 0 days 00:00:00 0                40                 103.156606   
                                  graph500-25 0 days 00:00:00 0                40                 227.895927   
                                  graph500-26 0 days 00:00:00 0                40                 491.334392   
                                  uniform-22  0 days 00:00:00 0                40                  22.539563   
                                  uniform-24  0 days 00:00:00 0                40                 105.765634   
                                  uniform-25  0 days 00:00:00 0                40                 230.742329   
                                  uniform-26  0 days 00:00:00 0                40                 479.670000   
csr                      gcc      dota-league 0 days 00:00:00 0                0                    1.235302   
                                  graph500-22 0 days 00:00:00 0                0                    1.639669   
                                  graph500-24 0 days 00:00:00 0                0                    2.509866   
                                  graph500-26 0 days 00:00:00 0                0                    5.025450   
                                  uniform-22  0 days 00:00:00 0                0                    1.610202   
                                  uniform-24  0 days 00:00:00 0                0                    2.402618   
                                  uniform-26  0 days 00:00:00 0                0                    4.180822   
                         n/a      dota-league 0 days 00:00:00 0                0                    1.007152   
                                  graph500-22 0 days 00:00:00 0                0                    1.330705   
                                  graph500-24 0 days 00:00:00 0                0                    2.171562   
                                  graph500-26 0 days 00:00:00 0                0                    5.257210   
                                  uniform-22  0 days 00:00:00 0                0                    1.327486   
                                  uniform-24  0 days 00:00:00 0                0                    2.218205   
                                  uniform-26  0 days 00:00:00 0                0                    4.088046   
csr-lcc                  gcc      dota-league 0 days 00:00:00 0                0                    1.234838   
                                  graph500-22 0 days 00:00:00 0                0                    1.620581   
                                  graph500-24 0 days 00:00:00 0                0                    2.492062   
                                  graph500-26 0 days 00:00:00 0                0                    5.478020   
                                  uniform-22  0 days 00:00:00 0                0                    1.604942   
                                  uniform-24  0 days 00:00:00 0                0                    2.366859   
                                  uniform-26  0 days 00:00:00 0                0                    4.206149   
                         n/a      dota-league 0 days 00:00:00 0                0                    1.008342   
                                  graph500-22 0 days 00:00:00 0                0                    1.360806   
                                  graph500-24 0 days 00:00:00 0                0                    2.264903   
                                  graph500-26 0 days 00:00:00 0                0                    5.231499   
                                  uniform-22  0 days 00:00:00 0          

If the compiler is null, these are the old results built with Clang v10.

# Results per platform

In [24]:
csr = analysis.loc[("csr-lcc", "gcc", slice(None))].droplevel([1,2,3])
graphmat = analysis.loc[("graphmat"), :].droplevel([0,2,3,4])
graphblas = analysis.loc[("graphblas"), :].droplevel([0,2,3,4])
graphone = analysis.loc[("g1_v4-ref-ignore-build")].droplevel([0,2,3,4])
livegraph = analysis.loc[("livegraph_ro", "gcc", slice(None), slice(None), slice(None), 20)].droplevel([1,2])
llama = analysis.loc[("llama6-ref", "gcc", slice(None), "00:00:10", 0, 16), :].droplevel([0,1,3,4,5]) # dynamic scheduling in OpenMP
stinger = analysis.loc[("stinger-ref"), :].droplevel([1,2,3])
teseo_dense_vertices = analysis.loc[("teseo-ta-lcc-dv.9", "gcc", slice(None), slice(None), slice(None), 40)].droplevel([1,2])
teseo_logical_vertices = analysis.loc[("teseo-ta-lcc.9", "gcc", slice(None), slice(None), slice(None), 40)].droplevel([1,2])

In [26]:
teseo_logical_vertices

,bfs,cdlp,lcc,pagerank,sssp,wcc
graph,,,,,,
dota-league,1.228841,3.366266,104.859715,1.912714,2.210030,1.231026
graph500-22,1.680622,9.668302,76.080428,3.718172,5.685777,2.468564
graph500-24,2.633830,42.946081,648.972667,13.578051,24.940906,7.259024
graph500-26,6.357447,200.776219,NaN,60.144979,126.020308,28.366381
uniform-22,1.661942,14.803705,4.976576,4.469878,7.590411,2.142333
uniform-24,2.536412,64.093727,21.534859,16.199022,32.005891,6.968144
uniform-26,4.800363,288.730586,96.694337,67.245159,148.872448,18.457637


Speed-up of the CSR w.r.t. GraphMat and GraphBlas:

In [32]:
graphmat/csr.loc[["graph500-24", "uniform-24"]]

,bfs,cdlp,lcc,pagerank,sssp,wcc
graph,,,,,,
graph500-24,5.069296,35.624549,NaN,25.299029,5.477749,56.768309
uniform-24,17.344928,40.951739,130.344056,53.691908,8.134029,70.002696


In [33]:
graphblas/csr

,bfs,cdlp,lcc,pagerank,sssp,wcc
graph,,,,,,
dota-league,0.114995,5.093734,1.596020,0.740346,0.944004,0.279460
graph500-22,0.617680,1.435310,2.720213,2.760402,0.852443,1.035538
graph500-24,0.740752,3.013259,2.948953,5.345627,1.562790,1.331261
graph500-25,NaN,NaN,NaN,NaN,NaN,NaN
graph500-26,4.509111,2.835986,2.794508,6.321525,1.709071,2.186516
uniform-22,0.305930,2.280305,2.076068,3.910229,1.158146,0.843411
uniform-24,0.510804,1.210632,3.041109,6.559962,2.174926,2.042284
uniform-25,NaN,NaN,NaN,NaN,NaN,NaN
uniform-26,3.829394,1.351287,3.851240,7.462300,2.582083,3.383026
